# 1. 음식 데이터 수집 및 레시피 크롤링

- 한식 이름 정의

In [ ]:
food_dict = {
    '돼지갈비구이': 0, '갈치구이': 1, '고등어구이': 2, '곱창구이': 3, '닭갈비': 4, 
    '백김치': 5, '떡갈비': 6, '불고기': 7, '삼겹살구이': 8, '장어구이': 9, '조개구이': 10,
    '떡국': 11, '무국': 12, '미역국': 13, '북엇국': 14, '시래기국': 15, '콩나물국': 16,
    '콩자반': 17, '편육': 18, '갓김치': 19, '깍두기': 20, '무생채': 21, '배추김치': 22,
    '오이소박이': 23, '총각김치': 24, '파김치': 25, '가지볶음': 26, '고사리나물': 27,
    '미역줄기볶음': 28, '숙주나물': 29, '시금치나물': 30, '애호박볶음': 31, '송편': 32,
    '막국수': 33, '물냉면': 34, '비빔냉면': 35, '수제비': 36, '열무국수': 37, '잔치국수': 38,
    '쫄면': 39, '칼국수': 40, '콩국수': 41, '꽈리고추무침': 42, '도토리묵': 43, '잡채': 44,
    '콩나물무침': 45, '김밥': 46, '김치볶음밥': 47, '누룽지': 48, '비빔밥': 49, '주먹밥': 50,
    '감자채볶음': 51, '두부김치': 52, '떡볶이': 53, '멸치볶음': 54, '소세지볶음': 55,
    '어묵볶음': 56, '오징어채볶음': 57, '제육볶음': 58, '주꾸미볶음': 59, '보쌈': 60,
    '수정과': 61, '식혜': 62, '간장게장': 63, '양념게장': 64, '깻잎장아찌': 65, '감자전': 66,
    '계란말이': 67, '김치전': 68, '파전': 69, '호박전': 70, '갈치조림': 71, '감자조림': 72,
    '고등어조림': 73, '두부조림': 74, '메추리알장조림': 75, '연근조림': 76, '전복죽': 77,
    '호박죽': 78, '김치찌개': 79, '닭계장': 80, '동태찌개': 81, '된장찌개': 82,
    '순두부찌개': 83, '갈비찜': 84, '김치찜': 85, '닭볶음탕': 86, '수육': 87, '순대': 88,
    '족발': 89, '찜닭': 90, '갈비탕': 91, '감자탕': 92, '매운탕': 93, '삼계탕': 94,
    '추어탕': 95, '약과': 96, '약식': 97, '한과': 98, '육회': 99, 'LA 갈비': 100,
    '등갈비구이': 101, '숯불닭갈비': 102, '만두국': 103, '김치콩나물국': 104, '신김치무국': 105
}

- 만개의레시피 크롤링 

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

def food_info(name):
    try:
        url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
        response = requests.get(url)
        response.raise_for_status() 
        
        soup = BeautifulSoup(response.text, 'html.parser')
        food_list = soup.find_all(attrs={'class':'common_sp_link'})
        
        if not food_list:
            print(f"{name}에 대한 음식을 찾을 수 없습니다.")
            return None
        
        food_id = None
        for food in food_list:
            food_name_element = food.find(attrs={'class': 'common_sp_caption_tit'})
            if food_name_element is not None:
                food_name = food_name_element.text.strip()
                if food_name == name:
                    food_id = food['href'].split('/')[-1]
                    break
        
        if food_id is None:
            food_id = food_list[0]['href'].split('/')[-1]
        
        new_url = f'https://www.10000recipe.com/recipe/{food_id}'
        new_response = requests.get(new_url)
        new_response.raise_for_status()
        
        soup = BeautifulSoup(new_response.text, 'html.parser')
        food_info = soup.find(attrs={'type':'application/ld+json'})
        
        if not food_info:
            print(f"{name}에 대한 레시피 데이터를 찾을 수 없습니다.")
            return None
        
        result = json.loads(food_info.text)
        ingredients = ', '.join(result.get('recipeIngredient', []))
        recipe = [f"{i+1}. {step.get('text', '')}" for i, step in enumerate(result.get('recipeInstructions', []))]
        
        res = {
            'name': name,
            'ingredients': ingredients,
            'recipe': recipe
        }
        return res
    
    except requests.exceptions.RequestException as e:
        print(f"HTTP 요청이 실패했습니다: {e}")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

- 엑셀파일로 레시피 저장

In [ ]:
import os

def save_multiple_foods_to_excel(food_dict, file_name, save_directory):
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    file_path = os.path.join(save_directory, file_name)
    all_data = []

    for food_name in food_dict.keys():
        info = food_info(food_name)
        if info is not None:
            recipe_steps = {f'단계 {i+1}': info['recipe'][i] for i in range(len(info['recipe']))}
            data = {
                '음식 이름': info['name'],
                '재료': info['ingredients']
            }
            data.update(recipe_steps) 
            all_data.append(data)
        else:
            print(f"{food_name}에 대한 정보를 찾을 수 없습니다.")
    
    df = pd.DataFrame(all_data)
    df.to_excel(file_path, index=False)
    print(f"{file_path}에 모든 음식 정보가 저장되었습니다.")

- 구글 번역 API를 사용한 레시피 번역

In [ ]:
from googletrans import Translator

def translate_to_english(text):
    translator = Translator()
    try:
        translated = translator.translate(text, src='ko', dest='en')
        return translated.text
    except Exception as e:
        print(f"번역 오류: {e}")
        return text

def translate_and_save_to_csv(file_name, translated_file_name):
    try:
        df = pd.read_excel(file_name)
        translated_df = df.applymap(lambda x: translate_to_english(str(x)) if isinstance(x, str) else x)
        translated_df.to_csv(translated_file_name, index=False, encoding='utf-8-sig')
        print(f"번역된 파일이 {translated_file_name}에 저장되었습니다.")
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_name}")
    except Exception as e:
        print(f"오류 발생: {e}")

- 번역되지 않은 부분 탐색

In [ ]:
import pandas as pd

def compare_and_print_untranslated(korean_file, english_file):
    korean_df = pd.read_excel(korean_file)
    english_df = pd.read_excel(english_file)
    
    if korean_df.shape != english_df.shape:
        print("한글 파일과 영어 파일의 크기가 다릅니다.")
        return

    for i in range(korean_df.shape[0]):
        for j in range(korean_df.shape[1]):
            if pd.isna(english_df.iloc[i, j]) or english_df.iloc[i, j] == '':
                print(f"번역되지 않은 부분: {korean_df.iloc[i, j]}")

- 최종 번역본 검수 및 저장

In [ ]:
def final_translation_output(translated_file, output_file):
    df_translated = pd.read_excel(translated_file)
    df_output = pd.read_excel(output_file)
    
    for row in range(len(df_translated)):
        for col in df_translated.columns:
            df_output.at[row, col] = df_translated.at[row, col]
    
    final_output_path = 'final_output_with_translation.xlsx'
    df_output.to_excel(final_output_path, index=False)
    print("번역된 데이터가 최종 파일에 저장되었습니다:", final_output_path)